# Network Operations
## Model Server

In [29]:
import nuclio

## Setup Environment

In [37]:
%nuclio config spec.build.baseImage = "mlrun/mlrun:dev"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun:latest'


In [2]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle
python -m pip install lightgbm

## Serving class

In [ ]:
# nuclio: start-code

In [3]:
import os
from cloudpickle import load
import numpy as np
from typing import List

In [52]:
class ClassifierModel():
    def __init__(self, name: str, model_dir: str):
        self.name = name
        self.model_dir = model_dir
        self.classifier = None
        self.ready = False

    def load(self):
        """Load model from storage.
        """        
        self.classifier = load(open(self.model_dir, 'rb'))
        self.ready = True
        
    def preprocess(self, r):
        return r
    
    def postprocess(self, r):
        return r

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        :param body : A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            if not self.ready:
                self.load()
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [ ]:
# nuclio: end-code

## Test

In [ ]:
import pandas as pd

# Set model
model_name = 'predictor'
model_path = '/User/demo-network-operations/artifacts/1cc3e2d1b5c54a3bb8dc68e102bbfcae/models/model.pkl'

# Load pre-processed data example
df = pd.read_parquet('/User/demo-network-operations/data/aggregate.pq')

# Set sample
sample = df.head(1).fillna(0).drop(columns=['is_error']).values.tolist()
msg = {'instances': sample}

## Test Locally

In [7]:
# Setup classifier
clf = ClassifierModel(name=model_name, model_dir=model_path)
clf.load()

/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.21.3 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [17]:
# Test
clf.predict(msg)

Python> 2020-03-13 06:42:19,384 [info] {'instances': [[78.39690454966423, 0.0, 0.0, 0.0, 0.0, 0.0, 251.55313092592772, 0.0, 0.0, 0.0, 0.0]]}


[0.0]

## Deploy to cluster

In [53]:
from mlrun import new_model_server, mount_v3io

# Setup the model server function
fn = new_model_server(name='failures-predictor',
                      model_class='ClassifierModel',
                      models={model_name: model_path})
fn.apply(mount_v3io())

# Deploy the model server
addr = fn.deploy(project='network-operations')

[mlrun] 2020-03-13 07:05:26,016 deploy started
[nuclio] 2020-03-13 07:06:26,702 (info) Build complete
[nuclio] 2020-03-13 07:06:36,821 (info) Function deploy complete
[nuclio] 2020-03-13 07:06:36,829 done updating failures-predictor, function address: 3.136.215.154:30837


In [54]:
import requests
import json
req = requests.post(f'{addr}/{model_name}/predict', data=json.dumps(msg))
req.__dict__

{'_content': b'[0.0]',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Server': 'nuclio', 'Date': 'Fri, 13 Mar 2020 07:06:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '5'},
 'raw': <urllib3.response.HTTPResponse at 0x7fed73442e48>,
 'url': 'http://3.136.215.154:30837/predictor/predict',
 'encoding': None,
 'history': [],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(0, 1, 70449),
 'request': <PreparedRequest [POST]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7fed7350e1d0>}